In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('/content/drive/MyDrive/DTS/Proa_ML_tensor/healthcare_project/dataset.csv')
desc_sym = pd.read_csv('/content/drive/MyDrive/DTS/Proa_ML_tensor/healthcare_project/symptom_Description.csv')
prec_sym = pd.read_csv('/content/drive/MyDrive/DTS/Proa_ML_tensor/healthcare_project/symptom_precaution.csv')
sym_wight = pd.read_csv('/content/drive/MyDrive/DTS/Proa_ML_tensor/healthcare_project/Symptom-severity.csv')

cleansing

In [3]:
#mengilangkan white space
dataset = dataset.apply(lambda x: x.str.strip())
dataset = dataset.replace('_',' ', regex=True).replace('  ',' ', regex=True).replace(' ofu',' of u', regex=True)
desc_sym = desc_sym.apply(lambda x: x.str.strip())
prec_sym = prec_sym.apply(lambda x: x.str.strip())

Split dataset (split in every disease)

In [4]:
from sklearn.model_selection import train_test_split
def df_split(dataset, test_size):
  df_train = pd.DataFrame()
  df_test = pd.DataFrame()
  for dis in dataset.Disease.unique().tolist():
    dataset_dis = dataset.loc[dataset.Disease == dis]
    train, test = train_test_split(dataset_dis, test_size=test_size, random_state=7)
    df_train = df_train.append(train)
    df_test = df_test.append(test)
  return df_train, df_test

In [5]:
train, test = df_split(dataset, 0.2)
print(len(dataset), len(train), len(test))

4920 3936 984


In [6]:
train.Disease.value_counts()

Fungal infection                          96
Hepatitis C                               96
Hepatitis E                               96
Alcoholic hepatitis                       96
Tuberculosis                              96
Common Cold                               96
Pneumonia                                 96
Dimorphic hemmorhoids(piles)              96
Heart attack                              96
Varicose veins                            96
Hypothyroidism                            96
Hyperthyroidism                           96
Hypoglycemia                              96
Osteoarthristis                           96
Arthritis                                 96
(vertigo) Paroymsal Positional Vertigo    96
Acne                                      96
Urinary tract infection                   96
Psoriasis                                 96
Hepatitis D                               96
Hepatitis B                               96
Allergy                                   96
hepatitis 

In [7]:
test.Disease.value_counts()

Fungal infection                          24
Hepatitis C                               24
Hepatitis E                               24
Alcoholic hepatitis                       24
Tuberculosis                              24
Common Cold                               24
Pneumonia                                 24
Dimorphic hemmorhoids(piles)              24
Heart attack                              24
Varicose veins                            24
Hypothyroidism                            24
Hyperthyroidism                           24
Hypoglycemia                              24
Osteoarthristis                           24
Arthritis                                 24
(vertigo) Paroymsal Positional Vertigo    24
Acne                                      24
Urinary tract infection                   24
Psoriasis                                 24
Hepatitis D                               24
Hepatitis B                               24
Allergy                                   24
hepatitis 

In [8]:
# save train test dataset
train.to_csv('train_dataset.csv', index=False)
test.to_csv('test_dataset.csv',  index=False)

##  feature extraction

onehot encode

In [9]:
from collections import Counter
import itertools

In [10]:
def df_symptoms_list(dataset):
    # symptoms to list
    dataset = pd.DataFrame({'Disease':dataset.Disease, 'Symptoms':dataset.drop(columns=['Disease']).values.tolist()})
    dataset['Symptoms'] = dataset.Symptoms.apply(lambda case: [symptom for symptom in case if str(symptom) != 'nan']) # remove nan value
    return dataset

In [11]:
# list all symptoms from train_dataset
def all_symptom(df_train_sym_list):
    all_sym = list(itertools.chain.from_iterable(df_train_sym_list.Symptoms))
    return list(dict.fromkeys(all_sym))

In [12]:
#one hot encoding symtomps
def onehot_sym(symptoms:list, all_symptom_data:list):
    matrix_sym = [0] * len(all_symptom_data) #inisiate onehot matrix
    for symptom in symptoms:
        try:
            index = all_symptom_data.index(symptom)
            matrix_sym[index] = 1
        except:pass # pass, when the symptom not in all_symptom_data
    return np.array(matrix_sym).reshape(1,-1)

In [13]:
def df_sym_onehot(dataset, all_symptom_data:list):
    # symptoms to onehot
    dataset['onehot_symtoms'] = dataset.Symptoms.apply(lambda x: onehot_sym(x, all_symptom_data))
    return dataset

In [14]:
def train_test(sym_train, sym_test):
    X_train = np.concatenate((np.array(sym_train.onehot_symtoms)))
    X_test = np.concatenate((np.array(sym_test.onehot_symtoms)))
    y_train = np.array(sym_train.Disease)
    y_test = np.array(sym_test.Disease)
    return X_train, X_test, y_train, y_test

In [15]:
def train_test(train, test):
    # symptoms to list
    train = df_symptoms_list(train)
    test = df_symptoms_list(test)

    # list all symptoms from train_dataset
    all_symptom_data = all_symptom(train)

    # symptoms to onehot
    df_train_onehot = df_sym_onehot(train, all_symptom_data)
    df_test_onehot = df_sym_onehot(test, all_symptom_data)

    # convert data into suitable model format
    X_train = np.concatenate((np.array(df_train_onehot.onehot_symtoms)))
    X_test = np.concatenate((np.array(df_test_onehot.onehot_symtoms)))
    y_train = np.array(df_train_onehot.Disease)
    y_test = np.array(df_test_onehot.Disease)
    return X_train, X_test, y_train, y_test, df_train_onehot, df_test_onehot

In [16]:
train = pd.read_csv('train_dataset.csv')
test = pd.read_csv('test_dataset.csv')
train.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin rash,nodal skin eruptions,dischromic patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,itching,skin rash,nodal skin eruptions,dischromic patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,skin rash,nodal skin eruptions,dischromic patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin rash,nodal skin eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin rash,nodal skin eruptions,dischromic patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
X_train, X_test, y_train, y_test, df_train, df_test= train_test(train, test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
df_train.head()

(3936, 131) (984, 131) (3936,) (984,)


,Disease,Symptoms,onehot_symtoms
0,Fungal infection,"[itching, skin rash, nodal skin eruptions, dis...","[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,Fungal infection,"[itching, skin rash, nodal skin eruptions, dis...","[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Fungal infection,"[itching, skin rash, nodal skin eruptions, dis...","[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,Fungal infection,"[itching, skin rash, nodal skin eruptions]","[[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,Fungal infection,"[itching, skin rash, nodal skin eruptions, dis...","[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


# Model